**Input size for both models: `(batch_size, 160, 192, 160, n_channels=1)`**

# Fake random dataset
shape: (n_samples, n_channels = 1, 160, 192, 160)

In [18]:
n_samples = 100 ## ?! 1000 is too large too alocate
target = np.random.random((n_samples, 1, 160, 192, 160))
background = np.random.random((n_samples, 1, 160, 192, 160))

# cVAE

In [1]:
# import os
# import sys
# SFCN_path = "../contrastive_vae-master" ## change to your own path that contain cVAE reposiotry code
# sys.path.append(SFCN_path)

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# from vae_utils import contrastive_vae, plot_latent_space, plot_latent_space4d, plot_clean_digits, plot_sweeps_mnist
# from sklearn.metrics import silhouette_score

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
## have some problems on tensorflow GPU (windows sys)
#tf.config.list_physical_devices('GPU')

## Build cVAE

- Notation diff from the papar: `z_` for salient feature, `s_` for irrelevant features
- Modify a bit from the original `contrastive_vae` function:
    - ? move batch size to input argument (why 100 in the original code?), and set in the input shape
    - set default disentangle value to True
    - ? `intermediate_dim` and `latent_dim` are two hyper-param to adjust
    - ? all layers to be fully connected -- tooooo computationally expensive?

In [15]:
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras import backend as K

## copy paste from celeb_utils.py
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

## modified from vae_utils.py
def my_contrastive_vae(input_dim=160 * 192 * 160, intermediate_dim=12, latent_dim=2, \
                       beta=1, disentangle=True, gamma=0, batch_size = 100):
    input_shape = (input_dim, )
    #batch_size = 100  ## ？why 100; move to input args
    tg_inputs = tf.keras.layers.Input(shape=input_shape, name='tg_input', batch_size=batch_size)
    bg_inputs = tf.keras.layers.Input(shape=input_shape, name='bg_input', batch_size=batch_size)
    
    if isinstance(intermediate_dim, int):
        intermediate_dim = [intermediate_dim]
    
    z_h_layers = []
    for dim in intermediate_dim:
        z_h_layers.append(tf.keras.layers.Dense(dim, activation='relu'))
    z_mean_layer = tf.keras.layers.Dense(latent_dim, name='z_mean')
    z_log_var_layer = tf.keras.layers.Dense(latent_dim, name='z_log_var')
    z_layer = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,), name='z')

    def z_encoder_func(inputs):
        z_h = inputs
        for z_h_layer in z_h_layers:
            z_h = z_h_layer(z_h)
        z_mean = z_mean_layer(z_h)
        z_log_var = z_log_var_layer(z_h)
        z = z_layer([z_mean, z_log_var])
        return z_mean, z_log_var, z

    s_h_layers = []
    for dim in intermediate_dim:
        s_h_layers.append(tf.keras.layers.Dense(dim, activation='relu'))
    s_mean_layer = tf.keras.layers.Dense(latent_dim, name='s_mean')
    s_log_var_layer = tf.keras.layers.Dense(latent_dim, name='s_log_var')
    s_layer = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,), name='s')

    def s_encoder_func(inputs):
        s_h = inputs
        for s_h_layer in s_h_layers:
            s_h = s_h_layer(s_h)
        s_mean = s_mean_layer(s_h)
        s_log_var = s_log_var_layer(s_h)
        s = s_layer([s_mean, s_log_var])
        return s_mean, s_log_var, s

    tg_z_mean, tg_z_log_var, tg_z = z_encoder_func(tg_inputs)
    tg_s_mean, tg_s_log_var, tg_s = s_encoder_func(tg_inputs)
    bg_s_mean, bg_s_log_var, bg_s = s_encoder_func(bg_inputs)

    z_encoder = tf.keras.models.Model(tg_inputs, [tg_z_mean, tg_z_log_var, tg_z], name='z_encoder')
    s_encoder = tf.keras.models.Model(tg_inputs, [tg_s_mean, tg_s_log_var, tg_s], name='s_encoder')

    # build decoder model
    cvae_latent_inputs = tf.keras.layers.Input(shape=(2 * latent_dim,), name='sampled')
    cvae_h = cvae_latent_inputs
    for dim in intermediate_dim:
        cvae_h = tf.keras.layers.Dense(dim, activation='relu')(cvae_h)
    cvae_outputs = tf.keras.layers.Dense(input_dim)(cvae_h)

    cvae_decoder = tf.keras.models.Model(inputs=cvae_latent_inputs, outputs=cvae_outputs, name='decoder')

    # decoder.summary()

    def zeros_like(x):
        return tf.zeros_like(x)

    tg_outputs = cvae_decoder(tf.keras.layers.concatenate([tg_z, tg_s], -1))
    zeros = tf.keras.layers.Lambda(zeros_like)(tg_z)
    bg_outputs = cvae_decoder(tf.keras.layers.concatenate([zeros, bg_s], -1))
    fg_outputs = cvae_decoder(tf.keras.layers.concatenate([tg_z, zeros], -1))

    cvae = tf.keras.models.Model(inputs=[tg_inputs, bg_inputs], 
                                 outputs=[tg_outputs, bg_outputs], 
                                 name='contrastive_vae')

    cvae_fg = tf.keras.models.Model(inputs=tg_inputs, 
                                    outputs=fg_outputs, 
                                    name='contrastive_vae_fg')

    # cvae.summary()
    if disentangle:
        discriminator = Dense(1, activation='sigmoid')
        
        z1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_z)
        z2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_z)
        s1 = Lambda(lambda x: x[:int(batch_size/2),:])(tg_s)
        s2 = Lambda(lambda x: x[int(batch_size/2):,:])(tg_s)
        q_bar = tf.keras.layers.concatenate(
            [tf.keras.layers.concatenate([s1, z2], axis=1),
            tf.keras.layers.concatenate([s2, z1], axis=1)],
            axis=0)
        q = tf.keras.layers.concatenate(
            [tf.keras.layers.concatenate([s1, z1], axis=1),
            tf.keras.layers.concatenate([s2, z2], axis=1)],
            axis=0)
        q_bar_score = discriminator(q_bar)
        q_score = discriminator(q)        
        tc_loss = K.log(q_score / (1 - q_score)) 
        
        discriminator_loss = - K.log(q_score) - K.log(1 - q_bar_score)

    reconstruction_loss = tf.keras.losses.mse(tg_inputs, tg_outputs)
    reconstruction_loss += tf.keras.losses.mse(bg_inputs, bg_outputs)
    reconstruction_loss *= input_dim


    kl_loss = 1 + tg_z_log_var - tf.keras.backend.square(tg_z_mean) - tf.keras.backend.exp(tg_z_log_var)
    kl_loss += 1 + tg_s_log_var - tf.keras.backend.square(tg_s_mean) - tf.keras.backend.exp(tg_s_log_var)
    kl_loss += 1 + bg_s_log_var - tf.keras.backend.square(bg_s_mean) - tf.keras.backend.exp(bg_s_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    if disentangle:
        cvae_loss = K.mean(reconstruction_loss) + beta*K.mean(kl_loss) + gamma * K.mean(tc_loss) + K.mean(discriminator_loss)
    else:
        cvae_loss = K.mean(reconstruction_loss) + K.mean(kl_loss)

    cvae.add_loss(cvae_loss)
    cvae.compile(optimizer='adam')
    
    return cvae, cvae_fg, z_encoder, s_encoder, cvae_decoder


batch_size = 32
## ? what for cvae_fg 
cvae, cvae_fg, z_encoder, s_encoder, cvae_decoder = my_contrastive_vae(
    input_dim=160 * 192 * 160, intermediate_dim=32, latent_dim=2, batch_size=batch_size)

In [16]:
cvae.summary()

Model: "contrastive_vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 tg_input (InputLayer)          [(32, 4915200)]      0           []                               
                                                                                                  
 dense_18 (Dense)               (32, 32)             157286432   ['tg_input[0][0]']               
                                                                                                  
 bg_input (InputLayer)          [(32, 4915200)]      0           []                               
                                                                                                  
 z_mean (Dense)                 (32, 2)              66          ['dense_18[0][0]']               
                                                                                    

## Reshape data to 1D
- ?? reshape

In [27]:
full_dim = 160 * 192 * 160
target_flat = target.reshape(-1, full_dim)
background_flat = background.reshape(-1, full_dim)

## Fit the model
- ?? in original code: no set aside validation data

In [30]:
## batch size specified while initializing the model
epochs = 10
history = cvae.fit([target_flat, background_flat], epochs=10, batch_size=batch_size, \
                   validation_data=([target_flat, background_flat], None), verbose=1) ## reset verbose for logging info

plt.plot(history.history['val_loss'])
min_loss = min(history.history['val_loss'])
plt.title('Min loss:' + str(min_loss))

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'contrastive_vae/decoder/dense_21/BiasAdd' defined at (most recent call last):
    File "D:\miniconda3\envs\ConGele\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\miniconda3\envs\ConGele\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\miniconda3\envs\ConGele\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\miniconda3\envs\ConGele\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "D:\miniconda3\envs\ConGele\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "D:\miniconda3\envs\ConGele\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "D:\miniconda3\envs\ConGele\lib\asyncio\base_events.py", line 570, in run

In [29]:
import gc

del target
del background
gc.collect()

14692